In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import time
import gym
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
class ReplayBuffer:
    def __init__(self, max_size=5e5):
        self.buffer = []
        self.max_size = int(max_size)
        self.size = 0
    
    def add(self, transition):
        self.size +=1
        # transiton is tuple of (state, action, reward, next_state, done)
        self.buffer.append(transition)
    
    def sample(self, batch_size):
        # delete 1/5th of the buffer when full
        if self.size > self.max_size:
            del self.buffer[0:int(self.size/5)]
            self.size = len(self.buffer)
        
        indexes = np.random.randint(0, len(self.buffer), size=batch_size)
        state, action, reward, next_state, done = [], [], [], [], []
        
        for i in indexes:
            s, a, r, s_, d = self.buffer[i]
            state.append(np.array(s, copy=False))
            action.append(np.array(a, copy=False))
            reward.append(np.array(r, copy=False))
            next_state.append(np.array(s_, copy=False))
            done.append(np.array(d, copy=False))
        
        return np.array(state), np.array(action), np.array(reward), np.array(next_state), np.array(done)
    

In [ ]:
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()
        
        self.l1 = nn.Linear(state_dim, 400)
        self.l2 = nn.Linear(400, 300)
        self.l3 = nn.Linear(300, action_dim)
        
        self.max_action = max_action
        
    def forward(self, state):
        a = F.relu(self.l1(state))
        a = F.relu(self.l2(a))
        a = torch.tanh(self.l3(a)) * self.max_action
        return a
        
class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()
        
        self.l1 = nn.Linear(state_dim + action_dim, 400)
        self.l2 = nn.Linear(400, 300)
        self.l3 = nn.Linear(300, 1)
        
    def forward(self, state, action):
        state_action = torch.cat([state, action], 1)
        
        q = F.relu(self.l1(state_action))
        q = F.relu(self.l2(q))
        q = self.l3(q)
        return q
    
class TD3:
    def __init__(self, lr, state_dim, action_dim, max_action):
        
        self.actor = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=lr)
        
        self.critic_1 = Critic(state_dim, action_dim).to(device)
        self.critic_1_target = Critic(state_dim, action_dim).to(device)
        self.critic_1_target.load_state_dict(self.critic_1.state_dict())
        self.critic_1_optimizer = optim.Adam(self.critic_1.parameters(), lr=lr)
        
        self.critic_2 = Critic(state_dim, action_dim).to(device)
        self.critic_2_target = Critic(state_dim, action_dim).to(device)
        self.critic_2_target.load_state_dict(self.critic_2.state_dict())
        self.critic_2_optimizer = optim.Adam(self.critic_2.parameters(), lr=lr)
        
        self.max_action = max_action
    
    def select_action(self, state):
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        return self.actor(state).cpu().data.numpy().flatten()
    
    def update(self, replay_buffer, n_iter, batch_size, gamma, polyak, policy_noise, noise_clip, policy_delay):
        
        for i in range(n_iter):
            # Sample a batch of transitions from replay buffer:
            state, action_, reward, next_state, done = replay_buffer.sample(batch_size)
            state = torch.FloatTensor(state).to(device)
            action = torch.FloatTensor(action_).to(device)
            reward = torch.FloatTensor(reward).reshape((batch_size,1)).to(device)
            next_state = torch.FloatTensor(next_state).to(device)
            done = torch.FloatTensor(done).reshape((batch_size,1)).to(device)
            
            # Select next action according to target policy:
            noise = torch.FloatTensor(action_).data.normal_(0, policy_noise).to(device)
            noise = noise.clamp(-noise_clip, noise_clip)
            next_action = (self.actor_target(next_state) + noise)
            next_action = next_action.clamp(-self.max_action, self.max_action)
            
            # Compute target Q-value:
            target_Q1 = self.critic_1_target(next_state, next_action)
            target_Q2 = self.critic_2_target(next_state, next_action)
            target_Q = torch.min(target_Q1, target_Q2)
            target_Q = reward + ((1-done) * gamma * target_Q).detach()
            
            # Optimize Critic 1:
            current_Q1 = self.critic_1(state, action)
            loss_Q1 = F.mse_loss(current_Q1, target_Q)
            self.critic_1_optimizer.zero_grad()
            loss_Q1.backward()
            self.critic_1_optimizer.step()
            
            # Optimize Critic 2:
            current_Q2 = self.critic_2(state, action)
            loss_Q2 = F.mse_loss(current_Q2, target_Q)
            self.critic_2_optimizer.zero_grad()
            loss_Q2.backward()
            self.critic_2_optimizer.step()
            
            # Delayed policy updates:
            if i % policy_delay == 0:
                # Compute actor loss:
                actor_loss = -self.critic_1(state, self.actor(state)).mean()
                
                # Optimize the actor
                self.actor_optimizer.zero_grad()
                actor_loss.backward()
                self.actor_optimizer.step()
                
                # Polyak averaging update:
                for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                    target_param.data.copy_( (polyak * target_param.data) + ((1-polyak) * param.data))
                
                for param, target_param in zip(self.critic_1.parameters(), self.critic_1_target.parameters()):
                    target_param.data.copy_( (polyak * target_param.data) + ((1-polyak) * param.data))
                
                for param, target_param in zip(self.critic_2.parameters(), self.critic_2_target.parameters()):
                    target_param.data.copy_( (polyak * target_param.data) + ((1-polyak) * param.data))
                    
                
    def save(self, directory, name):
        torch.save(self.actor.state_dict(), '%s/%s_actor.pth' % (directory, name))
        torch.save(self.actor_target.state_dict(), '%s/%s_actor_target.pth' % (directory, name))
        
        torch.save(self.critic_1.state_dict(), '%s/%s_crtic_1.pth' % (directory, name))
        torch.save(self.critic_1_target.state_dict(), '%s/%s_critic_1_target.pth' % (directory, name))
        
        torch.save(self.critic_2.state_dict(), '%s/%s_crtic_2.pth' % (directory, name))
        torch.save(self.critic_2_target.state_dict(), '%s/%s_critic_2_target.pth' % (directory, name))
        
    def load(self, directory, name):
        self.actor.load_state_dict(torch.load('%s/%s_actor.pth' % (directory, name), map_location=lambda storage, loc: storage))
        self.actor_target.load_state_dict(torch.load('%s/%s_actor_target.pth' % (directory, name), map_location=lambda storage, loc: storage))
        
        self.critic_1.load_state_dict(torch.load('%s/%s_crtic_1.pth' % (directory, name), map_location=lambda storage, loc: storage))
        self.critic_1_target.load_state_dict(torch.load('%s/%s_critic_1_target.pth' % (directory, name), map_location=lambda storage, loc: storage))
        
        self.critic_2.load_state_dict(torch.load('%s/%s_crtic_2.pth' % (directory, name), map_location=lambda storage, loc: storage))
        self.critic_2_target.load_state_dict(torch.load('%s/%s_critic_2_target.pth' % (directory, name), map_location=lambda storage, loc: storage))
        
        
    def load_actor(self, directory, name):
        self.actor.load_state_dict(torch.load('%s/%s_actor.pth' % (directory, name), map_location=lambda storage, loc: storage))
        self.actor_target.load_state_dict(torch.load('%s/%s_actor_target.pth' % (directory, name), map_location=lambda storage, loc: storage))
        
        
        
      
        

episode: 2256
episode length: 85
episode reward: 27.62357543348556
current timestep: 30399

episode: 2257
episode length: 86
episode reward: 28.253464308683494
current timestep: 30485

episode: 2258
episode length: 98
episode reward: 39.36948244703435
current timestep: 30583

episode: 2259
episode length: 89
episode reward: 31.415976916624807
current timestep: 30672

episode: 2260
episode length: 95
episode reward: 37.160364489841804
current timestep: 30767

episode: 2261
episode length: 84
episode reward: 27.21930539056115
current timestep: 30851

episode: 2262
episode length: 94
episode reward: 35.907474491769044
current timestep: 30945

episode: 2263
episode length: 81
episode reward: 24.82054050297668
current timestep: 31026

episode: 2264
episode length: 84
episode reward: 26.78520929929127
current timestep: 31110

episode: 2265
episode length: 85
episode reward: 27.333834718536455
current timestep: 31195

episode: 2266
episode length: 85
episode reward: 28.004934086562447
current

episode: 2344
episode length: 52
episode reward: -32.22569746458371
current timestep: 35979

episode: 2345
episode length: 52
episode reward: -33.08132891006094
current timestep: 36031

episode: 2346
episode length: 52
episode reward: -31.739953598425434
current timestep: 36083

episode: 2347
episode length: 51
episode reward: -29.888409257032
current timestep: 36134

episode: 2348
episode length: 53
episode reward: -39.533894372969606
current timestep: 36187

episode: 2349
episode length: 52
episode reward: -34.13455691440129
current timestep: 36239

episode: 2350
episode length: 49
episode reward: -27.358318452447637
current timestep: 36288

episode: 2351
episode length: 53
episode reward: -34.9330278587643
current timestep: 36341

episode: 2352
episode length: 52
episode reward: -33.0599857960234
current timestep: 36393

episode: 2353
episode length: 53
episode reward: -34.81001439014153
current timestep: 36446

episode: 2354
episode length: 51
episode reward: -30.21366771750571
cur

episode: 2432
episode length: 40
episode reward: -4.811576550462265
current timestep: 39621

episode: 2433
episode length: 34
episode reward: -5.074755141324133
current timestep: 39655

episode: 2434
episode length: 32
episode reward: -3.1455664038305056
current timestep: 39687

episode: 2435
episode length: 38
episode reward: -1.2438885044655252
current timestep: 39725

episode: 2436
episode length: 36
episode reward: -4.810783156109407
current timestep: 39761

episode: 2437
episode length: 34
episode reward: -0.7087613775379771
current timestep: 39795

episode: 2438
episode length: 34
episode reward: -4.125682355152814
current timestep: 39829

episode: 2439
episode length: 39
episode reward: -0.7738957333457873
current timestep: 39868

episode: 2440
episode length: 35
episode reward: -2.478669308536079
current timestep: 39903

episode: 2441
episode length: 35
episode reward: -2.193078309594314
current timestep: 39938

episode: 2442
episode length: 40
episode reward: -9.23576850357077

episode: 2521
episode length: 30
episode reward: -4.3668069098738105
current timestep: 42607

episode: 2522
episode length: 29
episode reward: -4.000957852700101
current timestep: 42636

episode: 2523
episode length: 29
episode reward: -5.54577382886571
current timestep: 42665

episode: 2524
episode length: 31
episode reward: -3.192056236724817
current timestep: 42696

episode: 2525
episode length: 29
episode reward: -4.558469964669623
current timestep: 42725

episode: 2526
episode length: 31
episode reward: -4.872890466984833
current timestep: 42756

episode: 2527
episode length: 30
episode reward: -3.822149375900315
current timestep: 42786

episode: 2528
episode length: 31
episode reward: -4.805949479812241
current timestep: 42817

episode: 2529
episode length: 30
episode reward: -3.617534897054562
current timestep: 42847

episode: 2530
episode length: 31
episode reward: -4.689415319977422
current timestep: 42878

episode: 2531
episode length: 31
episode reward: -3.8770426948023955
c

episode: 2609
episode length: 29
episode reward: -4.385732441274925
current timestep: 45192

episode: 2610
episode length: 28
episode reward: -5.057448160932712
current timestep: 45220

episode: 2611
episode length: 30
episode reward: -4.998414241078807
current timestep: 45250

episode: 2612
episode length: 29
episode reward: -5.679814149253946
current timestep: 45279

episode: 2613
episode length: 28
episode reward: -5.27663668584759
current timestep: 45307

episode: 2614
episode length: 27
episode reward: -4.712179378339327
current timestep: 45334

episode: 2615
episode length: 30
episode reward: -4.77369645407697
current timestep: 45364

episode: 2616
episode length: 30
episode reward: -4.010817007473582
current timestep: 45394

episode: 2617
episode length: 28
episode reward: -5.067401877192664
current timestep: 45422

episode: 2618
episode length: 29
episode reward: -5.697915187768609
current timestep: 45451

episode: 2619
episode length: 28
episode reward: -5.489834077580864
curr

...

episode: 10303
episode length: 46
episode reward: -26.58856471088184
current timestep: 396735

episode: 10304
episode length: 38
episode reward: -7.612588469536695
current timestep: 396773

episode: 10305
episode length: 36
episode reward: -7.723245773961862
current timestep: 396809

episode: 10306
episode length: 42
episode reward: -18.822400092784417
current timestep: 396851

episode: 10307
episode length: 34
episode reward: -7.998496194705078
current timestep: 396885

episode: 10308
episode length: 33
episode reward: -9.576440470755328
current timestep: 396918

episode: 10309
episode length: 37
episode reward: -7.512313649534285
current timestep: 396955

episode: 10310
episode length: 31
episode reward: -8.302435558616246
current timestep: 396986

episode: 10311
episode length: 38
episode reward: -7.142260338853103
current timestep: 397024

episode: 10312
episode length: 37
episode reward: -7.997192654243142
current timestep: 397061

episode: 10313
episode length: 47
episode reward:

episode: 10390
episode length: 39
episode reward: -6.00925568423161
current timestep: 400208

episode: 10391
episode length: 43
episode reward: -10.161964010478762
current timestep: 400251

episode: 10392
episode length: 39
episode reward: -7.016056271630471
current timestep: 400290

episode: 10393
episode length: 39
episode reward: -6.009663513553717
current timestep: 400329

episode: 10394
episode length: 41
episode reward: -6.966389667646848
current timestep: 400370

episode: 10395
episode length: 35
episode reward: -6.282162204473903
current timestep: 400405

episode: 10396
episode length: 41
episode reward: -6.701640192893034
current timestep: 400446

episode: 10397
episode length: 43
episode reward: -10.728480800675667
current timestep: 400489

episode: 10398
episode length: 42
episode reward: -9.35880135888188
current timestep: 400531

episode: 10399
episode length: 40
episode reward: -8.790680844421152
current timestep: 400571

episode: 10400
episode length: 45
episode reward: 

episode: 10476
episode length: 46
episode reward: -15.871394811842654
current timestep: 403986

episode: 10477
episode length: 46
episode reward: -17.486789929511303
current timestep: 404032

episode: 10478
episode length: 45
episode reward: -14.521767277947722
current timestep: 404077

episode: 10479
episode length: 47
episode reward: -17.307344889803673
current timestep: 404124

episode: 10480
episode length: 49
episode reward: -23.14307918128066
current timestep: 404173

episode: 10481
episode length: 44
episode reward: -13.783694064210945
current timestep: 404217

episode: 10482
episode length: 48
episode reward: -19.384872020211457
current timestep: 404265

episode: 10483
episode length: 46
episode reward: -15.970476517621337
current timestep: 404311

episode: 10484
episode length: 47
episode reward: -15.580792163683888
current timestep: 404358

episode: 10485
episode length: 43
episode reward: -11.280281978065226
current timestep: 404401

episode: 10486
episode length: 46
episode

episode: 10562
episode length: 49
episode reward: -14.63255338085812
current timestep: 407952

episode: 10563
episode length: 50
episode reward: -20.597699566684057
current timestep: 408002

episode: 10564
episode length: 49
episode reward: -14.909543414160622
current timestep: 408051

episode: 10565
episode length: 55
episode reward: -29.383205291427114
current timestep: 408106

episode: 10566
episode length: 47
episode reward: -15.431056953095213
current timestep: 408153

episode: 10567
episode length: 47
episode reward: -10.856964056726406
current timestep: 408200

episode: 10568
episode length: 50
episode reward: -18.297186759567822
current timestep: 408250

episode: 10569
episode length: 47
episode reward: -12.589855450660387
current timestep: 408297

episode: 10570
episode length: 51
episode reward: -19.83218256841332
current timestep: 408348

episode: 10571
episode length: 44
episode reward: -10.919470971131831
current timestep: 408392

episode: 10572
episode length: 49
episode 

KeyboardInterrupt: 

In [ ]:
######### Hyperparameters #########
env_name = "Walker2d-v2"
log_interval = 100           # print avg reward after interval
gamma = 0.99                # discount for future rewards
batch_size = 100            # num of transitions sampled from replay buffer
lr = 0.001
exploration_noise = 0.2 
polyak = 0.995              # target policy update parameter (1-tau)
policy_noise = 0.2          # target policy smoothing noise
noise_clip = 0.5
policy_delay = 2            # delayed policy updates parameter
max_episodes = 1000         # max num of episodes
max_timesteps = 5000        # max timesteps in one episode
directory = "/home/imnotrachit/Documents/Collage_stuff/sem_6/rl/endsem/td3/preTrained/" # save trained models
filename = "TD3_{}".format(env_name)
###################################

env = gym.make(env_name)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])

policy = TD3(lr, state_dim, action_dim, max_action)
replay_buffer = ReplayBuffer()

rewards = []
times=[]
avg_reward=0

# training procedure:
for episode in range(1, max_episodes+1):
    start_time = time.time()
    state = env.reset()
    ep_reward=0
    for t in range(max_timesteps):
        # select action and add exploration noise:
        action = policy.select_action(state)
        action = action + np.random.normal(0, exploration_noise, size=env.action_space.shape[0])
        action = action.clip(env.action_space.low, env.action_space.high)
        
        # take action in env:
        next_state, reward, done, _ = env.step(action)
        replay_buffer.add((state, action, reward, next_state, float(done)))
        state = next_state
        
        avg_reward += reward
        ep_reward += reward
        
        # if episode is done then update policy:
        if done or t==(max_timesteps-1):
            policy.update(replay_buffer, t, batch_size, gamma, polyak, policy_noise, noise_clip, policy_delay)
            end_time = time.time()
            episode_time = end_time - start_time
            break
    
    # logging updates:
    rewards.append(ep_reward)
    times.append(episode_time)
        
    if episode % log_interval == 0:
        avg_reward = int(avg_reward / log_interval)
        print("Episode: {}\tAverage Reward: {}".format(episode, avg_reward))
        avg_reward = 0
        
name = filename + '_solved'
policy.save(directory, name)


plt.figure(figsize=(12, 6))
plt.plot(rewards)
plt.xlabel('Episodes')
plt.ylabel('Rewards')
plt.title('Episode vs Reward')
plt.show()

In [ ]:
np.save('td3_reward.npy',rewards)
np.save('td3_time.npy',times)